# 1. 'simple' bivariate linear regression

Before we get started lets import some libraries that we will definately need:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 

Lets begin with a simple example. We have two variables (x and y), each with some scores:

In [ ]:
x = np.array([0,1,2,3,4,5,6,7,8,9])
y = np.array([1,3,2,5,7,8,8,9,10,12])

plt.scatter(x,y)

We want to find the best possible straight line to represent these points. That's the challenge in this chapter.

## 1.1. Finding the best line 'by hand'
Any straight line can be written as:
$y = b_0 + b_1*x$

The $b_0$ is known as the constant or intercept and the $b_1$ as the slope or gradient. Both $b_0$ and $b_1$ are sometimes also referred to as coefficients, and sometimes only $b_1$ is deemed a coefficient.

We want to choose $b_0$ and $b_1$ in such a way that it minimizes the total difference with the known points. However, not just the normal difference, but actually the squared difference.

We could keep on guessing but some smart people did some math for us and came up with two scary looking formulas:

\begin{equation*}
b_1=\frac{\bar{xy}-\bar{x}\times\bar{y}}{s_x^2}
\end{equation*}

And

\begin{equation*}
b_0=\bar{y}-\bar{x}\times b_0
\end{equation*}

A symbol with a bar on top, simply means average (mean). The $\bar{xy}$ is the mean of the x values multiplied with y.

So lets calculate these for our example:

In [ ]:
sx2 = x.var()
mxy = np.array(x*y).mean()
b1=(mxy-x.mean()*y.mean())/sx2
b1

In [ ]:
b0=y.mean()-b1*x.mean()
b0

Now lets calculate our predicted values with these values:

In [ ]:
myPrediction = b0+b1*x

plt.scatter(x,y, color='blue')
plt.plot(x,myPrediction, color='red')


Not bad. 

## 1.2. How good is the best?

How well does our prediction actually work. We could of course simply determine the mean of the differences (the so-called residuals):

In [ ]:
np.average(y-myPrediction)

That doesn't seem right. We are not far off with the prediction, but this seems ridiculous low. The reason are the negative values, we simply want the difference in absolute values:

In [ ]:
MAE = np.mean(np.absolute(y-myPrediction))
print(MAE)

This value is sometimes known as the Mean Absolute Error (MAE). As we saw in the previous session squaring in stead of absolute value is more common in statistics:

In [ ]:
MSE = np.mean((y-myPrediction)**2)
print(MSE)

This is the Mean Squared Error (MSE) and as with the standard deviation, we can take the square root out of this to get the Root Mean Squared Error.

In [ ]:
RMSE=MSE**(0.5)
print(RMSE)

However, more common to indicate how well a model is predicting the data a so-called coefficient of determination is calculated. This is usually written as $r^2$. You might recognize that $r$ from the previous session, it was the correlation coefficient. One way of calculating the determination coefficient is indeed by simply squaring the correlation coefficient.

The coefficient of determination will always be between 0 and 1. It is a percentage of the variance in the dependent variable (y) that is predictable from the independent variable(s) (x).

The formula for the correlation coefficient is usually given by:

\begin{equation*}
r=\frac{s_{xy}}{s_x\times s_y}
\end{equation*}

Here $s_{xy}$ is used to indicate the covariance, which in itself can be determined by:

\begin{equation*}
s_{xy}=\frac{\sum(x-\bar{x})\times(y-\bar{y})}{n-1}
\end{equation*}

I'm using here everywhere $s$ which usually is used for a so-called sample standard deviation. This divides by 'n'. However python more often uses $\sigma$ which is the population standard deviation. In this case it doesn't really matter since they actually will cancel each other out (the n-1 in the covariance will be cancelling the n-1 in the two standard deviations), and especially with big data using n or n - 1 will not lead to a big difference.

Okay, lets calculate that correlation coefficient and determination coefficient:

In [ ]:
covxy = np.sum((x-x.mean())*(y-y.mean()))/x.size
sX = x.std()
sY = y.std()

cor = covxy/(sX*sY)
print(cor)

det = cor**2
print(det)

The calculation above for the determination coefficient shows the link between the correl and the determination coefficient. However there are other formulas that lead to the same result.

The determination coefficient is a percentage. So we can also look at the total variation in the original values:

\begin{equation*}
SS_{tot}=\sum(y-\bar{y})^2
\end{equation*}

Look at how much variation there is left:

\begin{equation*}
SS_{res}=\sum(\hat{y}-y)^2
\end{equation*}

In this equation $\hat{y}$ are the predicted values. We then divide the two we get the percentage of unexplained variance:

\begin{equation*}
\frac{SS_{res}}{SS_{tot}}
\end{equation*}

Since the determinaton coefficient is the percentage of explained variance, we can simply now find the determination coefficient using:

\begin{equation*}
r^2=1-\frac{SS_{res}}{SS_{tot}}
\end{equation*}

Using Python we can check this. A small trick makes the calculations a little easier. We already have the standard deviation of y, and then we can use that $SS_y=s_y^2\times n$. So we get:

In [ ]:
1-np.sum((y-myPrediction)**2)/(sY**2*y.size)

Don't worry, you don't have to remember all those formulas. Numpy has you covered. It has a function to determine the correlation coefficient:

In [ ]:
np.corrcoef(x,y)

It returns a square matrix of 2x2. It shows the correlation coefficients between all possible pairs. So the 1 in the upper left corner is the correlation between x and x. The 0.97... is the correlation between x and y, and then in the next row we have the correlation between y and x, and finally between y and y. The diagonal will always be 1s.

Just to extract the correlation coefficient and get the determination coefficient is fairly easy now:

In [ ]:
np.corrcoef(x,y)[0,1]**2

Same as we had before, an extremely small difference which we'll consider a rounding error.

## 1.3. Using sklearn

Of course there have been others who have done this work for us. 
We could for example use sklearn (you'd have to install this first) and then you can import:

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn import metrics

It does require to reshape our x variable:

In [ ]:
xRes = x.reshape((-1,1))
yRes = y.reshape((-1,1))

To perform the regression analysis and saving the predicted results we can use:

In [ ]:
model = LinearRegression().fit(xRes,yRes)
yPred = model.predict(xRes)

As a reminder all the values we have calculated so far:

In [ ]:
print('The slope (b1): ',b1)
print('The intercept (b0): ',b0)
print('Mean Absolute Error:', MAE)
print('Mean Squared Error: ', MSE)
print('Root Mean Squared Error: ', RMSE)
print('Coefficient of determination: ',det)

Now lets see and compare the result with using sklearn

In [ ]:
b1V2=model.coef_[0]
print('The slope (b1): ',b1V2[0])
b0V2=model.intercept_
print('The intercept (b0): ',b0V2[0])
MAE2=metrics.mean_absolute_error(yRes,yPred)
print('Mean Absolute Error:', MAE2)
MSE2=metrics.mean_squared_error(yRes,yPred)
print('Mean Squared Error: ', MSE2)
RMSE2=metrics.mean_squared_error(yRes,yPred, squared=True)
print('Root Mean Squared Error: ', RMSE2)
det2=metrics.r2_score(yRes,yPred)
print('Coefficient of determination: ',det2)

A very small difference with our 'manual' formula for b1 and b0. We'll leave that as a rounding error :-)

You might notice I've used an index for the b1 coefficient, since we can actually also have multiple variables to use for our prediction. More on this later.


### 1.3.2. Exercise
On Moodle you will find a file Soccer2019C.csv. We want to predict the Overall score of players solely based on their age. To load the data we can use pandas:

In [ ]:
import pandas as pd

Once pandas is imported we can read a file as a pandas dataframe. If your file is in a separate folder 'data' we could use:

In [ ]:
soccerDF=pd.read_csv('data/Soccer2019C.csv')

Once the data is loaded you can get a quick overview using:

In [ ]:
soccerDF.head()

Your exercise is to find the linear regression equation to predict the Overall score, based on the age.

There are different ways you can do this:
1. Manually
2. Using the sklearn library
3. Using the statsmodels.api (not discussed yet)

You might have to convert the panda dataframe into a numpy array first. Try to find the regression equation with one (or even more to see if they all say the same).

Other things you could do if you think you're done....
* Add a visualisation
* Create a Python function to perform the manual calculations
* Find out which variable has the strongest determination coefficient to predict the Overall score
* Find out which two variables (one as predictor (x), one as predicted (y)) will have the strongest determination coefficient.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn import metrics

In [ ]:
age = soccerDF["Age"].to_numpy()
score = soccerDF["Overall"].to_numpy()

In [ ]:
np.corrcoef(age, score)

In [ ]:
x = age.reshape((-1,1))
y = score.reshape((-1,1))

model = LinearRegression().fit(x, y)
y_prediction = model.predict(x)

In [ ]:
print(y_prediction)

In [ ]:
plt.scatter(age, score, color="blue")
plt.plot(x, y_prediction, color="red")
plt.show()